In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import librosa
import glob

import sys
sys.path.insert(0,'..')
from mavd.data_generator import DataGenerator
from mavd.model import *
from mavd.callbacks import *

from keras.callbacks import CSVLogger
from keras.models import Model
from keras.layers import Concatenate,Dropout

import pickle

os.environ["CUDA_VISIBLE_DEVICES"]="0"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# files parameters
Nfiles = None
resume = False
load_subset = Nfiles

# audio parameters
sr = 22050
sequence_time = 1.0
sequence_hop_time = 0.5
audio_hop = 512
audio_win = 1024
n_fft = 1024
normalize_data = 'none' # para comparar loss con MST
get_annotations = True
mel_bands = 128

# training
learning_rate = 0.001
epochs = 201
batch_size = 64
sed_early_stopping = 100
epoch_limit = None
fit_verbose = True
fine_tuning = False

#model
large_cnn = True
frames = True

class_list1 = ['car','bus','motorcycle']
class_list2 = ['engine','breaks','wheel','compressor']
class_list3 = ['car/engine','car/wheel',
              'bus/engine', 'bus/breaks','bus/compressor','bus/wheel',
              'motorcycle/engine', 'motorcycle/breaks'] 

label_list = [class_list1,class_list2,class_list3]

# Create output folders
expfolder = '../exps/S-CNN_fine_tuning/'

audio_folder = '/data_ssd/users/pzinemanas/maestria/Slices/audio22050'
label_folder='/data_ssd/users/pzinemanas/maestria/Slices/labels'

In [9]:
params = {'sequence_time': sequence_time, 'sequence_hop_time':sequence_hop_time,
          'label_list':label_list, 'audio_hop':audio_hop, 'audio_win':audio_win,
          'n_fft':n_fft,'sr':sr,'mel_bands':mel_bands,'normalize':normalize_data, 'frames':frames,
          'get_annotations':get_annotations, 'dataset': 'MAVD'}

sequence_frames = int(np.ceil(sequence_time*sr/audio_hop))

# Datasets
partition = {}# IDs
labels = {}# Labels

test_files = sorted(glob.glob(os.path.join(audio_folder,'test', '*.wav')))
val_files = sorted(glob.glob(os.path.join(audio_folder,'validate', '*.wav')))

if load_subset is not None:
    test_files = test_files[:load_subset]
    val_files = val_files[:load_subset]

test_labels = {}
test_mel = {}
val_labels = {}
val_mel = {}
mel_basis = librosa.filters.mel(sr,n_fft,mel_bands,htk=True)
print('Founding scaler')
for n,id in enumerate(test_files):
    labels[id] = os.path.join(label_folder, 'test',os.path.basename(id).replace('.wav','.txt'))
    #train_mel[id]  = os.path.join(mel_folder, 'train',os.path.basename(id).replace('.wav','.npy.gz'))
for id in val_files:
    labels[id] = os.path.join(label_folder, 'validate',os.path.basename(id).replace('.wav','.txt'))

params['train'] = False
params['sequence_hop_time'] = sequence_time

# Generators
print('Making generators')
test_generator = DataGenerator(test_files, labels, **params)

validation_generator = DataGenerator(val_files, labels, **params)

print('Getting data')

_,_,x_val,y_val = validation_generator.return_all()
_,_,x_test,y_test = test_generator.return_all()

y_val_level1 = y_val[0]
y_val_level2 = y_val[1]
y_val_level3 = y_val[2]

y_test_level1 = y_test[0]
y_test_level2 = y_test[1]
y_test_level3 = y_test[2]

print('test',x_test.shape)    
print('val',x_val.shape)     

sequence_frames = x_val.shape[1]

# Build model

print('\nBuilding model...')

sequence_samples = int(sequence_time*sr)

model = cnn_fine_tuned(n_freq_cnn=mel_bands, n_frames_cnn=sequence_frames,large_cnn=large_cnn,
                      n_classes1=len(class_list1),n_classes2=len(class_list2),n_classes3=len(class_list3))

model.summary()

model.load_weights(os.path.join(expfolder,'weights_best.hdf5'))

# Fit model
print('\nTesting model...')

y_test_predicted = model.predict(x_test)
y_val_predicted = model.predict(x_val)

print('\nSaving results...')
data = {'val': {'y': y_val, 'predicted': y_val_predicted}, 'test': {'y': y_test, 'predicted': y_test_predicted}}
filename = os.path.join(expfolder,'results.pickle')
pickle.dump(data, open(filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

print('\nDone.')

Founding scaler
Making generators
Getting data
test (4990, 44, 128)
val (1995, 44, 128)

Building model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 44, 128)           0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 44, 128)           512       
_________________________________________________________________
lambda_3 (Lambda)            (None, 44, 128, 1)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 40, 124, 128)      3328      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 20, 62, 128)       0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 20, 62, 128)       512       
___________________________________